In [1]:
import pandas as pd
import numpy as np
from typing import Dict, Any, List
import warnings
import io
import base64
from IPython.display import display, HTML
warnings.filterwarnings('ignore')
# локальный файл
import script as sc
from phik import phik_matrix
from scipy import stats
import altair as alt
# from IPython.display import HTML

# Настройка Altair
alt.data_transformers.disable_max_rows() 
alt.data_transformers.enable('default')
alt.themes.enable('default')              

# палитра
colors = ["#59C5C7", "#157486", "#231F20", "#F16669", "#FFD73A", "#8B4A9C", "#2E8B57", "#FF6B35", "#4A90E2", "#A0522D", "#DC143C", "#32CD32", "#FF1493", "#00CED1", "#9932CC", "#FF8C00", "#228B22", "#FF69B4", "#20B2AA", "#8B008B", "#FFB6C1", "#3CB371", "#FF4500", "#4169E1", "#B8860B", "#FF6347", "#2F4F4F", "#DA70D6", "#00FA9A", "#CD5C5C", "#40E0D0", "#9370DB", "#FFA07A", "#008B8B", "#D2691E", "#FF7F50", "#6A5ACD", "#98FB98", "#F08080", "#87CEEB", "#DDA0DD", "#90EE90", "#F0E68C", "#E6E6FA", "#FFA500", "#AFEEEE", "#DB7093", "#F5DEB3", "#FF00FF", "#7FFFD4", "#C71585", "#FFDAB9", "#6495ED", "#DEB887", "#FF1493", "#B0E0E6", "#CD853F", "#FFEFD5", "#FF69B4", "#F0FFFF", "#D2B48C", "#FFFAF0", "#BA55D3", "#E0FFFF", "#A52A2A", "#F5F5DC", "#9400D3", "#FAF0E6", "#B22222", "#FDF5E6", "#4B0082", "#FFFACD", "#8B0000", "#F0FFF0", "#9932CC", "#F5FFFA", "#800000", "#F0F8FF", "#8A2BE2", "#FFF8DC", "#191970", "#FFEBCD", "#483D8B", "#FFE4E1", "#2F4F4F", "#DCDCDC", "#8FBC8F", "#F8F8FF", "#556B2F", "#FFFAFA", "#6B8E23", "#F0F0F0"]
color_scale = alt.Scale(range=colors)

In [2]:
# Словарь кодов стран – Визуализация
country_codes = {
    'ES': 'Испания', 'US': 'Соединенные Штаты Америки', 'CA': 'Канада', 'DE': 'Германия', 'GB': 'Великобритания', 'NG': 'Нигерия', 
    'IN': 'Индия', 'HK': 'Гонконг', 'PT': 'Португалия', 'NL': 'Нидерланды', 'CH': 'Швейцария', 'CF': 'Центральноафриканская Республика', 
    'FR': 'Франция', 'AU': 'Австралия', 'FI': 'Финляндия', 'UA': 'Украина', 'IE': 'Ирландия', 'IL': 'Израиль', 'GH': 'Гана', 'AT': 'Австрия',
    'CO': 'Колумбия', 'SG': 'Сингапур', 'SE': 'Швеция', 'SI': 'Словения', 'MX': 'Мексика', 'UZ': 'Узбекистан', 'BR': 'Бразилия', 
    'TH': 'Таиланд', 'HR': 'Хорватия', 'PL': 'Польша', 'KW': 'Кувейт', 'VN': 'Вьетнам', 'CY': 'Кипр', 'AR': 'Аргентина', 'AM': 'Армения', 
    'BA': 'Босния и Герцеговина', 'KE': 'Кения', 'GR': 'Греция', 'MK': 'Северная Македония', 'LV': 'Латвия', 'RO': 'Румыния', 
    'PK': 'Пакистан', 'IT': 'Италия', 'MA': 'Марокко', 'LT': 'Литва', 'BE': 'Бельгия', 'AS': 'Американское Самоа', 'IR': 'Иран', 
    'HU': 'Венгрия', 'SK': 'Словакия', 'CN': 'Китай', 'CZ': 'Чехия', 'CR': 'Коста-Рика', 'TR': 'Турция', 'CL': 'Чили', 'PR': 'Пуэрто-Рико', 
    'DK': 'Дания', 'BO': 'Боливия', 'PH': 'Филиппины', 'DO': 'Доминиканская Республика', 'EG': 'Египет', 'ID': 'Индонезия', 
    'AE': 'Объединенные Арабские Эмираты', 'MY': 'Малайзия', 'JP': 'Япония', 'EE': 'Эстония', 'HN': 'Гондурас', 'TN': 'Тунис', 
    'RU': 'Россия', 'DZ': 'Алжир', 'IQ': 'Ирак', 'BG': 'Болгария', 'JE': 'Джерси', 'RS': 'Сербия', 'NZ': 'Новая Зеландия', 'MD': 'Молдова', 
    'LU': 'Люксембург', 'MT': 'Мальта'}
# country_codes = {
#     'ES': 'Испания', 'US': 'Соединенные Штаты Америки', 'CA': 'Канада', 'DE': 'Германия', 'GB': 'Великобритания', 'NG': 'Нигерия',
#     'IN': 'Индия', 'HK': 'Гонконг', 'PT': 'Португалия', 'NL': 'Нидерланды', 'CH': 'Швейцария', 'CF': 'Центральноафриканская Республика',
#     'FR': 'Франция', 'AU': 'Австралия', 'FI': 'Финляндия', 'UA': 'Украина', 'IE': 'Ирландия', 'IL': 'Израиль', 'GH': 'Гана', 'AT': 'Австрия', 
#     'CO': 'Колумбия', 'SG': 'Сингапур', 'SE': 'Швеция','SI': 'Словения','MX': 'Мексика',
#     'UZ': 'Узбекистан',
#     'BR': 'Бразилия',
#     'TH': 'Таиланд',
#     'HR': 'Хорватия',
#     'PL': 'Польша',
#     'KW': 'Кувейт',
#     'VN': 'Вьетнам',
#     'CY': 'Кипр',
#     'AR': 'Аргентина',
#     'AM': 'Армения',
#     'BA': 'Босния и Герцеговина',
#     'KE': 'Кения',
#     'GR': 'Греция',
#     'MK': 'Северная Македония',
#     'LV': 'Латвия',
#     'RO': 'Румыния',
#     'PK': 'Пакистан',
#     'IT': 'Италия',
#     'MA': 'Марокко',
#     'LT': 'Литва',
#     'BE': 'Бельгия',
#     'AS': 'Американское Самоа',
#     'IR': 'Иран',
#     'HU': 'Венгрия',
#     'SK': 'Словакия',
#     'CN': 'Китай',
#     'CZ': 'Чехия',
#     'CR': 'Коста-Рика',
#     'TR': 'Турция',
#     'CL': 'Чили',
#     'PR': 'Пуэрто-Рико',
#     'DK': 'Дания',
#     'BO': 'Боливия',
#     'PH': 'Филиппины',
#     'DO': 'Доминиканская Республика',
#     'EG': 'Египет',
#     'ID': 'Индонезия',
#     'AE': 'Объединенные Арабские Эмираты',
#     'MY': 'Малайзия',
#     'JP': 'Япония',
#     'EE': 'Эстония',
#     'HN': 'Гондурас',
#     'TN': 'Тунис',
#     'RU': 'Россия',
#     'DZ': 'Алжир',
#     'IQ': 'Ирак',
#     'BG': 'Болгария',
#     'JE': 'Джерси',
#     'RS': 'Сербия',
#     'NZ': 'Новая Зеландия',
#     'MD': 'Молдова',
#     'LU': 'Люксембург',
#     'MT': 'Мальта'
# }

In [4]:
df = pd.read_csv('df.csv')

In [5]:
df.sample(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
1912,2022,MI,FT,Data Analyst,60000,USD,60000,US,100,US,M
2965,2022,SE,FT,Machine Learning Infrastructure Engineer,148800,USD,148800,US,100,US,M
766,2023,MI,FT,Data Scientist,45000,EUR,48289,ES,0,ES,M


In [6]:
analyzer = sc.DataAnalyzerTable(df)
analyzer.display_analysis_table()  

🔍 СВОДНАЯ ТАБЛИЦА АНАЛИЗА ДАННЫХ
📊 Датасет: 3,755 строк × 11 столбцов
💾 Размер в памяти: 1.64 MB


,Столбец,Описание,Кол-во значений,Типы данных,NaN (%),Кол-во уникальных,Примеры значений
0,work_year,Год выплаты зарплаты,3755,int,0.0,4,"Все (4): [2023, 2022, 2020, 2021]"
1,experience_level,Уровень опыта работы,3755,str,0.0,4,"Все (4): ['SE', 'MI', 'EN', 'EX']"
2,employment_type,Тип занятости,3755,str,0.0,4,"Все (4): ['FT', 'CT', 'FL', 'PT']"
3,job_title,Должность сотрудника,3755,str,0.0,93,Примеры (93 всего): ['Principal Data Scientist...
4,salary,Сумма зарплаты (GROSS),3755,int,0.0,815,"Примеры (815 всего): [80000, 30000, 25500, 175..."
5,salary_currency,Валюта зарплаты (ISO 4217),3755,str,0.0,20,"Примеры (20 всего): ['EUR', 'USD', 'INR', 'HKD..."
6,salary_in_usd,Зарплата в USD (целевая),3755,int,0.0,1035,"Примеры (1035 всего): [85847, 30000, 25500, 17..."
7,employee_residence,Страна сотрудника (ISO 3166),3755,str,0.0,78,"Примеры (78 всего): ['ES', 'US', 'CA', 'DE', '..."
8,remote_ratio,Доля удалённой работы,3755,int,0.0,3,"Все (3): [100, 0, 50]"
9,company_location,Страна компании,3755,str,0.0,72,"Примеры (72 всего): ['ES', 'US', 'CA', 'DE', '..."


In [7]:
print(f"Количество дубликатов: {df.duplicated().sum()}")

Количество дубликатов: 1171


In [8]:
df = df.drop_duplicates()
print(f"Количество дубликатов: {df.duplicated().sum()}")

Количество дубликатов: 0


## Данные готовы для дальнейшего анализа

# 
# 

### Распределение заработной платы по позициям с выявлением аномалий

In [9]:
# Боксплот зарплат по должностям с выбросами - вариант 1 (простой)
chart = alt.Chart(df).mark_boxplot(
    size=10,         
    outliers=True,         
    color='darkblue',           # Тёмно-синий цвет боксплота
    opacity=0.7                 # Прозрачность
).add_selection(
    alt.selection_interval()
).encode(
    x=alt.X('salary_in_usd:Q', 
           title='Зарплата (USD)', 
           scale=alt.Scale(zero=False)),
    y=alt.Y('job_title:O', 
           title='Должность', 
           sort=alt.EncodingSortField(field='salary_in_usd', op='median', order='descending'),
           axis=alt.Axis(labelLimit=200)),  # Увеличиваем лимит для длинных названий
    tooltip=['job_title:N', 'salary_in_usd:Q']
).properties(
    title='Распределение зарплат по должностям',
    width=600,
    height=1200
)
chart.show()  

alt.Chart(...)

In [10]:
# Топ должностей по медианной зарплате
top_jobs = df.groupby('job_title')['salary_in_usd'].agg(['median', 'mean', 'count']).reset_index()
top_jobs = top_jobs.sort_values('median', ascending=False)
print("Топ-5 должностей по медианной зарплате:")
for i, row in top_jobs.head().iterrows():
    print(f"{row['job_title']}: ${row['median']:,.0f} (медиана), ${row['mean']:,.0f} (среднее)")

Топ-5 должностей по медианной зарплате:
Data Science Tech Lead: $375,000 (медиана), $375,000 (среднее)
Cloud Data Architect: $250,000 (медиана), $250,000 (среднее)
Data Lead: $212,500 (медиана), $212,500 (среднее)
Data Analytics Lead: $211,254 (медиана), $211,254 (среднее)
Head of Data: $202,500 (медиана), $183,858 (среднее)


In [11]:
print("\nВыбросы в заработной плате:")
for job in df['job_title'].unique():
    job_data = df[df['job_title'] == job]['salary_in_usd']
    Q1 = job_data.quantile(0.25)
    Q3 = job_data.quantile(0.75)
    IQR = Q3 - Q1
    outliers = job_data[(job_data < Q1 - 1.5*IQR) | (job_data > Q3 + 1.5*IQR)]
    if len(outliers) > 0:
        print(f"  {job}: {len(outliers)} выбросов (max: ${outliers.max():,.0f})")


Выбросы в заработной плате:
  Principal Data Scientist: 1 выбросов (max: $416,000)
  Data Scientist: 4 выбросов (max: $412,000)
  Applied Scientist: 1 выбросов (max: $350,000)
  Data Analyst: 7 выбросов (max: $430,967)
  Analytics Engineer: 2 выбросов (max: $289,800)
  Machine Learning Engineer: 1 выбросов (max: $342,300)
  Data Engineer: 5 выбросов (max: $324,000)
  Data Architect: 2 выбросов (max: $376,080)
  Research Scientist: 2 выбросов (max: $450,000)
  Data Analytics Manager: 1 выбросов (max: $204,500)
  Applied Data Scientist: 1 выбросов (max: $380,000)
  Machine Learning Researcher: 1 выбросов (max: $50,000)
  Applied Machine Learning Scientist: 1 выбросов (max: $423,000)
  BI Developer: 1 выбросов (max: $197,000)
  Machine Learning Software Engineer: 2 выбросов (max: $375,000)
  Machine Learning Developer: 1 выбросов (max: $180,000)
  Lead Data Engineer: 1 выбросов (max: $276,000)


# 
# 

### ВЛИЯНИЕ УДАЛЕННОЙ РАБОТЫ НА ЗАРПЛАТУ

In [12]:
# Корреляция между удаленной работой и зарплатой
correlation = df['remote_ratio'].corr(df['salary_in_usd'])
print(f"Корреляция между удаленной работой и зарплатой: {correlation:.3f}")

# График 2: Зарплата vs удаленная работа
remote_labels = {0: 'Офис (0%)', 50: 'Гибрид (50%)', 100: 'Удаленно (100%)'}
df['remote_label'] = df['remote_ratio'].map(remote_labels)

chart2 = alt.Chart(df).mark_boxplot(size=80).encode(
    x=alt.X('remote_label:O', title='Тип работы', sort=['Офис (0%)', 'Гибрид (50%)', 'Удаленно (100%)']),
    y=alt.Y('salary_in_usd:Q', title='Зарплата (USD)', scale=alt.Scale(zero=False)),
    color=alt.Color('remote_label:N', scale=alt.Scale(range=colors[:3]), legend=None),
    tooltip=['remote_label:N', 'salary_in_usd:Q']
).properties(
    title='Зарплата vs Тип работы',
    width=500,
    height=400
)

chart2.show()

# Статистика по типам работы
remote_stats = df.groupby('remote_ratio')['salary_in_usd'].agg(['mean', 'median', 'count'])
print("\nСтатистика по типам работы:")
for ratio, stats in remote_stats.iterrows():
    label = remote_labels[ratio]
    print(f"  {label}: медиана ${stats['median']:,.0f}, среднее ${stats['mean']:,.0f}, количество {stats['count']}")


Корреляция между удаленной работой и зарплатой: -0.085


alt.Chart(...)


Статистика по типам работы:
  Офис (0%): медиана $137,500, среднее $143,690, количество 1186.0
  Гибрид (50%): медиана $63,312, среднее $78,487, количество 187.0
  Удаленно (100%): медиана $130,000, среднее $131,822, количество 1211.0


### Офисный формат показывает наиболее высокие зарплаты по всем ключевым показателям:

- Медиана: 137,000 (офис) vs 130,000 (удаленка) vs 63,000 (гибрид)
- Q3: 182,000 (офис) vs 175,000 (удаленка) vs 100,000 (гибрид)

##### Удаленный формат незначительно уступает офисному в верхних квартилях, но имеет более широкий разброс (Q1 = 84,000 против 100,000 в офисе).
##### Гибридный формат существенно отстает - медиана почти в 2 раза ниже офисного формата.
Статистическая значимость:
Различия в медианах составляют:

Офис vs Удаленка: 7,000 (5%)
Офис vs Гибрид: 74,000 (54%)
Удаленка vs Гибрид: 67,000 (52%)

# 
# 

### Зависимость зарплаты от размера компании

In [13]:
size_labels = {'S': 'Малая', 'M': 'Средняя', 'L': 'Большая'}
df['size_label'] = df['company_size'].map(size_labels)

chart3 = alt.Chart(df).mark_boxplot(size=80).encode(
    x=alt.X('size_label:O', title='Размер компании', sort=['Малая', 'Средняя', 'Большая']),
    y=alt.Y('salary_in_usd:Q', title='Зарплата (USD)', scale=alt.Scale(zero=False)),
    color=alt.Color('size_label:N', scale=alt.Scale(range=colors[3:6]), legend=None),
    tooltip=['size_label:N', 'salary_in_usd:Q']
).properties(
    title='Зарплата vs Размер компании',
    width=500,
    height=400
)

chart3.show()

# Статистика 
size_stats = df.groupby('company_size')['salary_in_usd'].agg(['mean', 'median', 'count'])
print("Статистика по размерам компаний:")
for size, stats in size_stats.iterrows():
    label = size_labels[size]
    print(f"  {label}: медиана ${stats['median']:,.0f}, среднее ${stats['mean']:,.0f}")

alt.Chart(...)

Статистика по размерам компаний:
  Большая: медиана $100,000, среднее $113,202
  Средняя: медиана $137,068, среднее $141,475
  Малая: медиана $62,726, среднее $78,364


### Средние компании показывают наилучшие результаты по зарплатам:

- Самая высокая медиана (137 000 против 63 000 у малых и 100 000 у больших)
- Самый высокий Q1 (99 000), что означает, что даже 25% самых низкооплачиваемых сотрудников получают больше, чем в других типах компаний
- Самый высокий Q3 (180 000)

### Малые компании демонстрируют наибольший разброс:

- Широкий диапазон зарплат от 5 600 до 416 000
- Низкая медиана (63 000), но высокий максимум
- Это может указывать на присутствие как стартапов с ограниченным бюджетом, так и успешных технологических компаний

### Большие компании показывают умеренные результаты:

- Медиана выше, чем у малых компаний, но ниже, чем у средних
- Относительно стабильное распределение зарплат

# 
# 

### АНАЛИЗ ПО СТРАНАМ

In [14]:
# Популярность стран (количество вакансий)
country_counts = df['company_location'].value_counts().head(8).reset_index()
country_counts.columns = ['country', 'count']
# Добавляем русские названия стран
country_counts['country_ru'] = country_counts['country'].map(country_codes).fillna(country_counts['country'])

chart4 = alt.Chart(country_counts).mark_bar().encode(
    x=alt.X('count:Q', title='Количество вакансий'),
    y=alt.Y('country_ru:O', title='Страна', sort='-x'),
    color=alt.Color('count:Q', scale=alt.Scale(scheme='viridis'), legend=None),
    tooltip=['country_ru:N', 'count:Q']
).properties(
    title='🌍 Популярность стран (количество вакансий)',
    width=600,
    height=400
)
chart4.show()

alt.Chart(...)

In [15]:
# Медианная зарплата по странам
country_salary = df.groupby('company_location')['salary_in_usd'].median().sort_values(ascending=False).head(8).reset_index()
country_salary.columns = ['country', 'median_salary']
# Добавляем русские названия стран
country_salary['country_ru'] = country_salary['country'].map(country_codes).fillna(country_salary['country'])

chart5 = alt.Chart(country_salary).mark_bar().encode(
    x=alt.X('median_salary:Q', title='Медианная зарплата (USD)'),
    y=alt.Y('country_ru:O', title='Страна', sort='-x'),
    color=alt.Color('median_salary:Q', scale=alt.Scale(scheme='plasma'), legend=None),
    tooltip=['country_ru:N', 'median_salary:Q']
).properties(
    title='💰 Медианная зарплата по странам',
    width=600,
    height=400
)
chart5.show()

alt.Chart(...)

In [54]:
print("Топ-5 стран по количеству вакансий:")
for _, row in country_counts.head().iterrows():
    print(f"  {row['country_ru']}: {row['count']} вакансий")

print("\nТоп-5 стран по медианной зарплате:")
for _, row in country_salary.head().iterrows():
    print(f"  {row['country_ru']}: ${row['median_salary']:,.0f}")

Топ-5 стран по количеству вакансий:
  Соединенные Штаты Америки: 1929 вакансий
  Великобритания: 155 вакансий
  Канада: 83 вакансий
  Индия: 57 вакансий
  Германия: 55 вакансий

Топ-5 стран по медианной зарплате:
  Израиль: $271,446
  Пуэрто-Рико: $167,500
  Соединенные Штаты Америки: $145,885
  Новая Зеландия: $125,000
  Канада: $120,000


In [17]:
for _, row in country_counts.iterrows():
    if row['country_ru'] == 'Израиль':
        print(f"  {row['country_ru']}: {row['count']} вакансий")

### Объем vs. Оплата
США - главный рынок для поиска работы (массовый спрос)
Израиль - нишевый рынок с премиальной оплатой (дефицит специалистов)
Англоязычные страны (США, Великобритания, Канада) концентрируют основную часть вакансий
Малые рынки (Пуэрто-Рико, Новая Зеландия) могут предложить высокие зарплаты при ограниченных возможностях

**Для массового поиска работы - США, для максимальной зарплаты - специализированные рынки типа Израиля (если есть соответствующая квалификация).**

# 
# 

### МЕДИАННАЯ ЗАРПЛАТА ПО ДОЛЖНОСТЯМ

In [32]:
job_median = df.groupby('job_title')['salary_in_usd'].median().reset_index()
job_median.columns = ['job_title', 'median_salary']
# визуализация
chart = alt.Chart(job_median.sort_values('median_salary', ascending=False).head()).mark_bar().encode(
    x=alt.X('median_salary:Q', title='Медианная зарплата (USD)'),
    y=alt.Y('job_title:O', title='Должность', sort='-x'),
    color=alt.Color('median_salary:Q', scale=alt.Scale(scheme='blues'), legend=None),
    tooltip=['job_title:N', 'median_salary:Q']
).properties(
    title='Медианная зарплата по должностям',
    width=450,
    height=150
)

In [53]:
result_df = display_df.sort_values('median_salary', ascending=False).reset_index(drop=True)
result_df.index = result_df.index + 1  

display_table = result_df[['job_title', 'formatted_salary']].rename(columns={
    'job_title': 'Профессия',
    'formatted_salary': 'Медианная зарплата'
})

# Настраиваем стили для прокрутки
def make_scrollable_table(df, height="235px"):
    return HTML(f"""
    <div style="height: {height}; overflow: auto; border: 1px solid #ccc;">
        {df.to_html(classes='table table-striped', table_id='scrollable-table')}
    </div>
    <style>
        #scrollable-table {{
            margin: 0;
            font-family: Arial, sans-serif;
        }}
        #scrollable-table th {{
            position: sticky;
            top: 0;
            background-color: #f8f9fa;
            z-index: 10;
            font-weight: bold;
            text-align: center;
        }}
        #scrollable-table td {{
            padding: 8px 12px;
        }}
        #scrollable-table tbody tr:hover {{
            background-color: #e8f4fd;
        }}
    </style>
    """)

make_scrollable_table(display_table)

,Профессия,Медианная зарплата
1,Data Science Tech Lead,"$375,000"
2,Cloud Data Architect,"$250,000"
3,Data Lead,"$212,500"
4,Data Analytics Lead,"$211,254"
5,Head of Data,"$202,500"
6,Principal Data Engineer,"$192,500"
7,Applied Scientist,"$192,000"
8,Principal Machine Learning Engineer,"$190,000"
9,Data Infrastructure Engineer,"$183,655"
10,Machine Learning Software Engineer,"$181,800"


In [33]:
# 
chart.show()

alt.Chart(...)

## Доминирование лидерских позиций - Data Science Tech Lead значительно опережает остальные должности ($375,000), что на 50% выше следующей позиции.
#### Премиум за техническое лидерство - топ-6 позиций представлены исключительно senior и lead ролями, подчеркивая высокую ценность опыта и управленческих навыков.
Градация по специализации:

- Техническое лидерство (Tech Lead) - максимум
- Архитектурные роли (Cloud Data Architect) - $250K

- Управленческие позиции (Data Lead, Head of Data) - $200-210K
- Инженерные lead-роли (Principal Data Engineer) - около $190K
  

***
Рынок четко премирует комбинацию технической экспертизы и лидерских компетенций, создавая значительный разрыв между топ-менеджментом и рядовыми специалистами.
***

# 
# 

### Изменение зарплат с 2020 по 2023

In [56]:
import pandas as pd
import numpy as np
import altair as alt
from scipy import stats

# Предполагаем, что у вас уже есть DataFrame df с данными о зарплатах

# 1. ОБЩИЙ АНАЛИЗ РЫНКА
print("="*60)
print("📊 ОБЩИЙ АНАЛИЗ ИЗМЕНЕНИЙ РЫНКА ЗАРПЛАТ 2020-2023")
print("="*60)

# Медианная зарплата по годам (общий тренд)
market_trend = df.groupby('work_year')['salary_in_usd'].agg(['median', 'mean', 'std', 'count']).reset_index()
market_2020 = market_trend[market_trend['work_year'] == 2020].iloc[0] if len(market_trend[market_trend['work_year'] == 2020]) > 0 else None
market_2023 = market_trend[market_trend['work_year'] == 2023].iloc[0] if len(market_trend[market_trend['work_year'] == 2023]) > 0 else None

if market_2020 is not None and market_2023 is not None:
    median_change = (market_2023['median'] - market_2020['median']) / market_2020['median'] * 100
    mean_change = (market_2023['mean'] - market_2020['mean']) / market_2020['mean'] * 100
    
    print(f"🌍 ОБЩИЙ РЫНОК:")
    print(f"  Медианная зарплата: ${market_2020['median']:,.0f} → ${market_2023['median']:,.0f} ({median_change:+.1f}%)")
    print(f"  Средняя зарплата: ${market_2020['mean']:,.0f} → ${market_2023['mean']:,.0f} ({mean_change:+.1f}%)")
    print(f"  Количество позиций: {market_2020['count']} → {market_2023['count']} ({(market_2023['count'] - market_2020['count']) / market_2020['count'] * 100:+.1f}%)")

# 2. АНАЛИЗ ПО ДОЛЖНОСТЯМ С СТАТИСТИЧЕСКОЙ ЗНАЧИМОСТЬЮ
print(f"\n📈 ДЕТАЛЬНЫЙ АНАЛИЗ ПО ДОЛЖНОСТЯМ:")
print("-"*60)

# Получаем данные только для должностей, которые есть и в 2020, и в 2023
salary_2020 = df[df['work_year'] == 2020].groupby('job_title')['salary_in_usd'].agg(['median', 'count', 'std']).reset_index()
salary_2023 = df[df['work_year'] == 2023].groupby('job_title')['salary_in_usd'].agg(['median', 'count', 'std']).reset_index()

# Объединяем данные
salary_comparison = pd.merge(salary_2020, salary_2023, on='job_title', suffixes=('_2020', '_2023'))

# Рассчитываем изменения
salary_comparison['change_pct'] = (salary_comparison['median_2023'] - salary_comparison['median_2020']) / salary_comparison['median_2020'] * 100
salary_comparison['change_abs'] = salary_comparison['median_2023'] - salary_comparison['median_2020']

# Фильтруем только должности с достаточным количеством наблюдений
min_observations = 5  # минимум наблюдений в каждом году
reliable_jobs = salary_comparison[
    (salary_comparison['count_2020'] >= min_observations) & 
    (salary_comparison['count_2023'] >= min_observations)
].copy()

print(f"Анализируем {len(reliable_jobs)} должностей с минимум {min_observations} наблюдениями в каждом году")

# 3. КАТЕГОРИЗАЦИЯ ИЗМЕНЕНИЙ
print(f"\n🎯 КАТЕГОРИЗАЦИЯ ИЗМЕНЕНИЙ:")
print("-"*40)

# Определяем категории изменений
def categorize_change(change_pct):
    if change_pct >= 50:
        return "🚀 Взлет"
    elif change_pct >= 20:
        return "📈 Сильный рост"
    elif change_pct >= 10:
        return "⬆️ Умеренный рост"
    elif change_pct >= -10:
        return "🔄 Стабильность"
    elif change_pct >= -20:
        return "⬇️ Умеренное снижение"
    else:
        return "📉 Сильное снижение"

reliable_jobs['category'] = reliable_jobs['change_pct'].apply(categorize_change)

# Считаем распределение по категориям
category_stats = reliable_jobs['category'].value_counts()
print("Распределение должностей по категориям изменений:")
for category, count in category_stats.items():
    percentage = count / len(reliable_jobs) * 100
    print(f"  {category}: {count} должностей ({percentage:.1f}%)")

# 4. ТОП-РОСТА И ПАДЕНИЯ
print(f"\n🏆 ТОП-10 ДОЛЖНОСТЕЙ ПО РОСТУ:")
print("-"*50)
top_growth = reliable_jobs.nlargest(10, 'change_pct')
for _, row in top_growth.iterrows():
    print(f"  {row['job_title'][:40]:<40} {row['change_pct']:+6.1f}% (${row['median_2020']:,.0f} → ${row['median_2023']:,.0f})")

print(f"\n📉 ТОП-10 ДОЛЖНОСТЕЙ ПО СНИЖЕНИЮ:")
print("-"*50)
top_decline = reliable_jobs.nsmallest(10, 'change_pct')
for _, row in top_decline.iterrows():
    print(f"  {row['job_title'][:40]:<40} {row['change_pct']:+6.1f}% (${row['median_2020']:,.0f} → ${row['median_2023']:,.0f})")

# 5. АНАЛИЗ ВОЛАТИЛЬНОСТИ РЫНКА
print(f"\n📊 АНАЛИЗ ВОЛАТИЛЬНОСТИ РЫНКА:")
print("-"*40)

# Рассчитываем статистики по изменениям
change_stats = reliable_jobs['change_pct'].describe()
print(f"Среднее изменение: {change_stats['mean']:+.1f}%")
print(f"Медианное изменение: {change_stats['50%']:+.1f}%")
print(f"Стандартное отклонение: {change_stats['std']:.1f}%")
print(f"Диапазон изменений: {change_stats['min']:+.1f}% до {change_stats['max']:+.1f}%")

# 6. ВИЗУАЛИЗАЦИЯ ОБЩИХ ТРЕНДОВ
print(f"\n📈 СОЗДАНИЕ ВИЗУАЛИЗАЦИЙ...")

# График 1: Распределение изменений
change_distribution = alt.Chart(reliable_jobs).mark_bar().encode(
    x=alt.X('change_pct:Q', bin=alt.Bin(step=10), title='Изменение зарплаты (%)'),
    y=alt.Y('count()', title='Количество должностей'),
    color=alt.Color('category:N', title='Категория изменения')
).properties(
    title='Распределение изменений зарплат по должностям (2020-2023)',
    width=600,
    height=300
)

# График 2: Scatter plot изменений
scatter_chart = alt.Chart(reliable_jobs).mark_circle(size=100).encode(
    x=alt.X('median_2020:Q', title='Медианная зарплата 2020 (USD)', scale=alt.Scale(zero=False)),
    y=alt.Y('change_pct:Q', title='Изменение (%)', scale=alt.Scale(zero=False)),
    color=alt.Color('category:N', title='Категория'),
    size=alt.Size('count_2020:Q', title='Количество наблюдений 2020'),
    tooltip=['job_title:N', 'median_2020:Q', 'median_2023:Q', 'change_pct:Q']
).properties(
    title='Зависимость роста зарплат от базового уровня',
    width=600,
    height=400
)

# График 3: Топ растущих и падающих
top_changes = pd.concat([
    reliable_jobs.nlargest(5, 'change_pct'),
    reliable_jobs.nsmallest(5, 'change_pct')
])

bar_chart = alt.Chart(top_changes).mark_bar().encode(
    y=alt.Y('job_title:N', sort='-x', title='Должность'),
    x=alt.X('change_pct:Q', title='Изменение (%)'),
    color=alt.Color('change_pct:Q', 
                   scale=alt.Scale(scheme='redyellowgreen', reverse=True),
                   title='Изменение (%)')
).properties(
    title='Топ-5 растущих и падающих должностей',
    width=600,
    height=300
)

print("Графики созданы! Используйте следующие команды для отображения:")
print("change_distribution.show()  # Распределение изменений")
print("scatter_chart.show()        # Scatter plot")
print("bar_chart.show()           # Топ изменений")

# 7. СВОДКА И ВЫВОДЫ
print(f"\n🎯 КЛЮЧЕВЫЕ ВЫВОДЫ:")
print("="*50)
print(f"1. Общий рынок: медианная зарплата изменилась на {median_change:+.1f}%")
print(f"2. Большинство должностей ({category_stats.get('🔄 Стабильность', 0) + category_stats.get('⬆️ Умеренный рост', 0)}) показали стабильность или умеренный рост")
print(f"3. Самый большой рост: {top_growth.iloc[0]['job_title']} (+{top_growth.iloc[0]['change_pct']:.1f}%)")
print(f"4. Самое большое снижение: {top_decline.iloc[0]['job_title']} ({top_decline.iloc[0]['change_pct']:.1f}%)")
print(f"5. Волатильность рынка: стандартное отклонение {change_stats['std']:.1f}%")

# Экспорт результатов для дальнейшего анализа
print(f"\n💾 Сохранение результатов...")
reliable_jobs.to_csv('salary_analysis_2020_2023.csv', index=False)
print("Результаты сохранены в 'salary_analysis_2020_2023.csv'")

# 8. ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ - ВЗВЕШЕННЫЕ ИЗМЕНЕНИЯ
print(f"\n⚖️ ВЗВЕШЕННЫЙ АНАЛИЗ РЫНКА:")
print("-"*40)

# Взвешиваем изменения по количеству позиций
reliable_jobs['weighted_change'] = reliable_jobs['change_pct'] * reliable_jobs['count_2023']
total_positions_2023 = reliable_jobs['count_2023'].sum()
market_weighted_change = reliable_jobs['weighted_change'].sum() / total_positions_2023

print(f"Взвешенное изменение рынка (по количеству позиций): {market_weighted_change:+.1f}%")

# Анализ по уровням зарплат
def salary_tier(salary):
    if salary >= 200000:
        return "Высокий (200k+)"
    elif salary >= 100000:
        return "Средний-высокий (100-200k)"
    elif salary >= 50000:
        return "Средний (50-100k)"
    else:
        return "Низкий (<50k)"

reliable_jobs['salary_tier_2020'] = reliable_jobs['median_2020'].apply(salary_tier)
tier_analysis = reliable_jobs.groupby('salary_tier_2020')['change_pct'].agg(['mean', 'median', 'count']).round(1)
print(f"\nИзменения по уровням зарплат 2020:")
print(tier_analysis)

# 9. АНАЛИЗ НОВЫХ И ИСЧЕЗНУВШИХ ДОЛЖНОСТЕЙ
print(f"\n🆕 АНАЛИЗ ПОЯВЛЕНИЯ/ИСЧЕЗНОВЕНИЯ ДОЛЖНОСТЕЙ:")
print("-"*50)

all_jobs_2020 = set(df[df['work_year'] == 2020]['job_title'].unique())
all_jobs_2023 = set(df[df['work_year'] == 2023]['job_title'].unique())

new_jobs = all_jobs_2023 - all_jobs_2020
disappeared_jobs = all_jobs_2020 - all_jobs_2023

print(f"Новые должности в 2023 ({len(new_jobs)}):")
for job in sorted(list(new_jobs)[:10]):  # показываем первые 10
    new_job_salary = df[(df['work_year'] == 2023) & (df['job_title'] == job)]['salary_in_usd'].median()
    print(f"  {job}: ${new_job_salary:,.0f}")

print(f"\nИсчезнувшие должности ({len(disappeared_jobs)}):")
for job in sorted(list(disappeared_jobs)[:10]):  # показываем первые 10
    old_job_salary = df[(df['work_year'] == 2020) & (df['job_title'] == job)]['salary_in_usd'].median()
    print(f"  {job}: ${old_job_salary:,.0f}")

# 10. ФИНАЛЬНАЯ СВОДКА
print(f"\n📋 ИТОГОВЫЙ ОТЧЕТ:")
print("="*60)
print(f"📊 Проанализировано должностей: {len(reliable_jobs)}")
print(f"📈 Общее изменение рынка: {median_change:+.1f}% (медиана), {market_weighted_change:+.1f}% (взвешенное)")
print(f"🎯 Должностей с ростом >20%: {len(reliable_jobs[reliable_jobs['change_pct'] > 20])}")
print(f"📉 Должностей со снижением >10%: {len(reliable_jobs[reliable_jobs['change_pct'] < -10])}")
print(f"🆕 Новых должностей: {len(new_jobs)}")
print(f"🗑️ Исчезнувших должностей: {len(disappeared_jobs)}")

# Возвращаем данные для дальнейшего использования
print(f"\n🔧 Переменные для дальнейшего анализа:")
print("- reliable_jobs: DataFrame с анализом изменений")
print("- change_distribution: график распределения")
print("- scatter_chart: scatter plot")
print("- bar_chart: график топ изменений")
print("- market_weighted_change: взвешенное изменение рынка")
print("- tier_analysis: анализ по уровням зарплат")

📊 ОБЩИЙ АНАЛИЗ ИЗМЕНЕНИЙ РЫНКА ЗАРПЛАТ 2020-2023
🌍 ОБЩИЙ РЫНОК:
  Медианная зарплата: $74,130 → $140,000 (+88.9%)
  Средняя зарплата: $93,353 → $146,998 (+57.5%)
  Количество позиций: 75.0 → 1156.0 (+1441.3%)

📈 ДЕТАЛЬНЫЙ АНАЛИЗ ПО ДОЛЖНОСТЯМ:
------------------------------------------------------------
Анализируем 3 должностей с минимум 5 наблюдениями в каждом году

🎯 КАТЕГОРИЗАЦИЯ ИЗМЕНЕНИЙ:
----------------------------------------
Распределение должностей по категориям изменений:
  🚀 Взлет: 3 должностей (100.0%)

🏆 ТОП-10 ДОЛЖНОСТЕЙ ПО РОСТУ:
--------------------------------------------------
  Data Analyst                             +210.5% ($34,784 → $108,000)
  Data Scientist                           +139.1% ($62,726 → $150,000)
  Data Engineer                            +101.0% ($72,134 → $145,000)

📉 ТОП-10 ДОЛЖНОСТЕЙ ПО СНИЖЕНИЮ:
--------------------------------------------------
  Data Engineer                            +101.0% ($72,134 → $145,000)
  Data Scientist        

In [55]:
# yearly_salary = df.groupby(['work_year', 'job_title'])['salary_in_usd'].median().reset_index()

# chart7 = alt.Chart(yearly_salary).mark_line(point=True, strokeWidth=3).encode(
#     x=alt.X('work_year:O', title='Год'),
#     y=alt.Y('salary_in_usd:Q', title='Медианная зарплата (USD)', scale=alt.Scale(zero=False)),
#     color=alt.Color('job_title:N', scale=color_scale, title='Должность'),
#     tooltip=['work_year:O', 'job_title:N', 'salary_in_usd:Q']
# ).properties(
#     title='📈 Изменение медианных зарплат по годам',
#     width=700,
#     height=400
# )

# chart7.show()

# # Сравнение 2020 и 2023
# salary_2020 = df[df['work_year'] == 2020].groupby('job_title')['salary_in_usd'].median()
# salary_2023 = df[df['work_year'] == 2023].groupby('job_title')['salary_in_usd'].median()
# salary_change = ((salary_2023 - salary_2020) / salary_2020 * 100).fillna(0)

# print("Изменение зарплат с 2020 по 2023:")
# for job, change in salary_change.sort_values(ascending=False).items():
#     if not pd.isna(change):
#         print(f"  {job}: {change:+.1f}%")


alt.Chart(...)

Изменение зарплат с 2020 по 2023:
  ML Engineer: +1027.4%
  AI Scientist: +403.8%
  Computer Vision Engineer: +254.2%
  Machine Learning Infrastructure Engineer: +242.0%
  Data Analyst: +210.5%
  Data Scientist: +139.1%
  Data Engineer: +101.0%
  Data Science Consultant: +72.9%
  Product Data Analyst: +25.9%
  Machine Learning Engineer: +4.5%
  Data Specialist: +0.0%
  Head of Data: +0.0%
  Deep Learning Engineer: +0.0%
  Deep Learning Researcher: +0.0%
  Data Science Lead: +0.0%
  ETL Engineer: +0.0%
  Data Science Engineer: +0.0%
  Financial Data Analyst: +0.0%
  Data Strategist: +0.0%
  AI Developer: +0.0%
  Head of Data Science: +0.0%
  Insight Analyst: +0.0%
  AI Programmer: +0.0%
  Lead Data Scientist: +0.0%
  MLOps Engineer: +0.0%
  Machine Learning Manager: +0.0%
  Machine Learning Research Engineer: +0.0%
  Machine Learning Researcher: +0.0%
  Machine Learning Software Engineer: +0.0%
  NLP Engineer: +0.0%
  Research Engineer: +0.0%
  Software Data Engineer: +0.0%
  Lead Data 

In [57]:
# g. Портрет успешного работника
print("\n🎯 Ж. ПОРТРЕТ УСПЕШНОГО РАБОТНИКА")
print("-"*45)

# Найдем характеристики топ-10% самых высокооплачиваемых работников
top_10_percent = df[df['salary_in_usd'] >= df['salary_in_usd'].quantile(0.9)]

print("Портрет топ-10% самых высокооплачиваемых работников:")
print(f"Средняя зарплата: ${top_10_percent['salary_in_usd'].mean():,.0f}")
print(f"Самая популярная должность: {top_10_percent['job_title'].mode().iloc[0]}")
print(f"Самая популярная страна проживания: {top_10_percent['employee_residence'].mode().iloc[0]}")
print(f"Самая популярная страна работы: {top_10_percent['company_location'].mode().iloc[0]}")
print(f"Самый популярный тип работы: {top_10_percent['remote_ratio'].mode().iloc[0]}% удаленно")
print(f"Самый популярный размер компании: {size_labels[top_10_percent['company_size'].mode().iloc[0]]}")
print(f"Самый популярный уровень опыта: {top_10_percent['experience_level'].mode().iloc[0]}")

# График 8: Характеристики успешных работников
success_remote = top_10_percent['remote_label'].value_counts().reset_index()
success_remote.columns = ['remote_type', 'count']

chart8 = alt.Chart(success_remote).mark_arc(innerRadius=50).encode(
    theta=alt.Theta('count:Q'),
    color=alt.Color('remote_type:N', scale=alt.Scale(range=colors[:3]), title='Тип работы'),
    tooltip=['remote_type:N', 'count:Q']
).properties(
    title='🎯 Тип работы успешных работников',
    width=400,
    height=400
)

chart8.show()



🎯 Ж. ПОРТРЕТ УСПЕШНОГО РАБОТНИКА
---------------------------------------------
Портрет топ-10% самых высокооплачиваемых работников:
Средняя зарплата: $261,870
Самая популярная должность: Data Engineer
Самая популярная страна проживания: US
Самая популярная страна работы: US
Самый популярный тип работы: 0% удаленно
Самый популярный размер компании: Средняя
Самый популярный уровень опыта: SE


alt.Chart(...)

In [ ]:
# h. Работа и проживание в разных регионах
print("\n🌐 З. РАБОТА И ПРОЖИВАНИЕ В РАЗНЫХ РЕГИОНАХ")
print("-"*45)

# Анализ географической мобильности
different_countries = df[df['employee_residence'] != df['company_location']]
same_countries = df[df['employee_residence'] == df['company_location']]

print(f"Работают в другой стране: {len(different_countries)} ({len(different_countries)/len(df)*100:.1f}%)")
print(f"Работают в стране проживания: {len(same_countries)} ({len(same_countries)/len(df)*100:.1f}%)")

# График 9: Географическая мобильность
mobility_df = pd.DataFrame({
    'category': ['Та же страна', 'Другая страна'],
    'count': [len(same_countries), len(different_countries)]
})

chart9 = alt.Chart(mobility_df).mark_bar().encode(
    x=alt.X('category:O', title='Тип работы'),
    y=alt.Y('count:Q', title='Количество работников'),
    color=alt.Color('category:N', scale=alt.Scale(range=colors[:2]), legend=None),
    tooltip=['category:N', 'count:Q']
).properties(
    title='🌐 Географическая мобильность',
    width=500,
    height=400
)

chart9.show()

# Анализ по странам проживания
residence_stats = df.groupby('employee_residence').agg({
    'salary_in_usd': 'count',
    'remote_ratio': lambda x: (x == 100).sum()
}).rename(columns={'salary_in_usd': 'total', 'remote_ratio': 'remote_100'})
residence_stats['remote_percent'] = residence_stats['remote_100'] / residence_stats['total'] * 100

print("\nТоп стран по количеству работников и % удаленной работы:")
top_residence = residence_stats.sort_values('total', ascending=False).head()
for country, stats in top_residence.iterrows():
    print(f"  {country}: {stats['total']} работников, {stats['remote_percent']:.1f}% полностью удаленно")

In [ ]:
# i. Дополнительные инсайты
print("\n✨ И. ДОПОЛНИТЕЛЬНЫЕ ИНСАЙТЫ")
print("-"*45)

# График 10: Распределение зарплат по уровню опыта
experience_labels = {'EN': 'Junior', 'MI': 'Middle', 'SE': 'Senior', 'EX': 'Executive'}
df['exp_label'] = df['experience_level'].map(experience_labels)

chart10 = alt.Chart(df).mark_boxplot(size=80).encode(
    x=alt.X('exp_label:O', title='Уровень опыта', sort=['Junior', 'Middle', 'Senior', 'Executive']),
    y=alt.Y('salary_in_usd:Q', title='Зарплата (USD)', scale=alt.Scale(zero=False)),
    color=alt.Color('exp_label:N', scale=color_scale, legend=None),
    tooltip=['exp_label:N', 'salary_in_usd:Q']
).properties(
    title='🎓 Зарплата по уровню опыта',
    width=500,
    height=400
)

chart10.show()

# График 11: Корреляционная матрица (упрощенная версия)
df_numeric = df.copy()
df_numeric['remote_ratio_num'] = df_numeric['remote_ratio']
df_numeric['company_size_num'] = df_numeric['company_size'].map({'S': 1, 'M': 2, 'L': 3})
df_numeric['experience_num'] = df_numeric['experience_level'].map({'EN': 1, 'MI': 2, 'SE': 3, 'EX': 4})

# Создаем scatter plot matrix для показа корреляций
chart11 = alt.Chart(df_numeric).mark_circle(size=60, opacity=0.6).encode(
    x=alt.X('experience_num:Q', title='Уровень опыта (числовой)'),
    y=alt.Y('salary_in_usd:Q', title='Зарплата (USD)'),
    color=alt.Color('company_size_num:Q', scale=alt.Scale(scheme='viridis'), title='Размер компании'),
    tooltip=['experience_num:Q', 'salary_in_usd:Q', 'company_size_num:Q']
).properties(
    title='🔍 Корреляция: Опыт vs Зарплата vs Размер компании',
    width=600,
    height=400
)

chart11.show()

# График 12: Интерактивное распределение зарплат
chart12 = alt.Chart(df).mark_bar(binExtent=[df['salary_in_usd'].min(), df['salary_in_usd'].max()]).encode(
    alt.X('salary_in_usd:Q', bin=alt.Bin(maxbins=50), title='Зарплата (USD)'),
    alt.Y('count()', title='Частота'),
    color=alt.value(colors[0]),
    tooltip=['count()']
).properties(
    title='📊 Распределение зарплат',
    width=600,
    height=400
)

# Добавляем линии для среднего и медианы
mean_line = alt.Chart(pd.DataFrame({'mean': [df['salary_in_usd'].mean()]})).mark_rule(
    color=colors[1], strokeDash=[5, 5], size=2
).encode(x='mean:Q')

median_line = alt.Chart(pd.DataFrame({'median': [df['salary_in_usd'].median()]})).mark_rule(
    color=colors[2], strokeDash=[5, 5], size=2
).encode(x='median:Q')

chart12_combined = (chart12 + mean_line + median_line).resolve_scale(color='independent')
chart12_combined.show()

# График 13: Scatter plot - Год × Удаленка × Зарплата
chart13 = alt.Chart(df).mark_circle(size=100, opacity=0.7).encode(
    x=alt.X('work_year:O', title='Год'),
    y=alt.Y('remote_ratio:Q', title='Удаленная работа (%)'),
    color=alt.Color('job_title:N', scale=color_scale, title='Должность'),
    size=alt.Size('salary_in_usd:Q', scale=alt.Scale(range=[50, 400]), title='Зарплата'),
    tooltip=['work_year:O', 'remote_ratio:Q', 'salary_in_usd:Q', 'job_title:N']
).properties(
    title='🚀 Анализ: Год × Удаленка × Зарплата',
    width=700,
    height=400
)

chart13.show()

# График 14: Тепловая карта зарплат по странам и должностям (упрощенная)
country_job_salary = df.groupby(['company_location', 'job_title'])['salary_in_usd'].median().reset_index()

chart14 = alt.Chart(country_job_salary).mark_rect().encode(
    x=alt.X('job_title:O', title='Должность'),
    y=alt.Y('company_location:O', title='Страна'),
    color=alt.Color('salary_in_usd:Q', scale=alt.Scale(scheme='viridis'), title='Медианная зарплата'),
    tooltip=['company_location:N', 'job_title:N', 'salary_in_usd:Q']
).properties(
    title='🌍 Медианная зарплата: Страна × Должность',
    width=700,
    height=400
)

chart14.show()

print("\n🔍 КЛЮЧЕВЫЕ ИНСАЙТЫ:")
print("1. Наибольший разброс зарплат наблюдается в топовых позициях")
print("2. Удаленная работа слабо коррелирует с уровнем зарплаты")
print("3. Размер компании влияет на зарплату больше, чем удаленная работа")
print("4. Опыт работы - самый значимый фактор для зарплаты")
print("5. Географическая мобильность довольно высока в IT-сфере")
print(f"6. Средняя зарплата в выборке: ${df['salary_in_usd'].mean():,.0f}")
print(f"7. Медианная зарплата в выборке: ${df['salary_in_usd'].median():,.0f}")

print("\n🎯 РЕКОМЕНДАЦИИ ДЛЯ КАРЬЕРНОГО РОСТА:")
print("• Фокусируйтесь на получении опыта - это ключевой фактор роста зарплаты")
print("• Рассмотрите работу в крупных компаниях для более высокой оплаты")
print("• Специализируйтесь в востребованных областях (ML, Data Engineering)")
print("• Рассмотрите возможности работы в высокооплачиваемых странах")
print("• Удаленная работа не снижает зарплату, но и не увеличивает значительно")

print("\n✨ ПРЕИМУЩЕСТВА ALTAIR:")
print("• Декларативный синтаксис на основе Grammar of Graphics")
print("• Автоматическая интерактивность (зум, панорамирование, выделение)")
print("• Интеграция с Jupyter Notebook и веб-приложениями")
print("• Простота создания сложных многослойных визуализаций")
print("• Экспорт в различные форматы (HTML, PNG, SVG, PDF)")
print("• Отличная производительность для больших датасетов")